# Чат-бот в 30 строчек кода

Чат-боты — один из самых популярных трендов последних лет. И неудивительно: мы не расстаемся со смартфонами, при этом большая часть экранного времени уходит на соцсети и мессенджеры. Это значит, что общаться с клиентами и пользователями удобнее всего именно там. Несмотря на то, что это большая и нетривиальная тема, для создания простейшего чат-бота в телеграме нужно не больше 30-35 строчек кода!

## Telegram API

Как и у любого уважающего себя сервиса, у телеграма есть API *(application program interface)* — специальный набор инструментов для взаимодействия с программой, а не с живым пользователем. Вот [официальная страница](https://core.telegram.org/bots/api) с  его документацией. 

Для популярных API обычно можно найти «обертки» на разных языках программирования. Обертка — это набор функций, которые собирают запросы, отправляют их, получают и интерпретируют ответы и т.д. — иными словами, облегчают работу с API. Существует довольно много питоновских оберток для телеграма: 
* [python-telegram-bot](https://github.com/python-telegram-bot/python-telegram-bot)
* [telepot](https://github.com/nickoala/telepot)
* [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI). 

Мы будем работать с последней из них — `pyTelegramBotAPI`. Эту библиотеку необходимо установить, написав в терминале `pip install pyTelegramBotAPI` или запустив следующую ячейку.

In [75]:
!pip install pyTelegramBotAPI

## API Token

Чтобы создать бота, нам необходимо получить уникальный токен. Как это сделать?

1. Если нет аккаунта в телеграме, то нужно зарегистрироваться — https://web.telegram.org (для этого нужно ввести номер телефона, получить смс с кодом и ввести код на странице подтверждения номера)

2. Затем нужно начать чат с крестным отцом всех ботов — https://t.me/BotFather.

  * Отправляем ему команду `/newbot`
  * Вводим имя бота — то, что будет отображаться как имя всем, кто с нашим ботом потом будет общаться
  * Вводим юзернейм бота — он должен заканчиваться на `bot` или `Bot` и быть уникальным.
  * Получаем API токен, уникальный для нашего бота. Если у вас несколько ботов, то у каждого будет свой API-токен!

<img src="./img/botfather.png" width="550" align="left">
<img src="./img/get_token.png" width="550" align="left">

**NB!** Логины, пароли, секретные токены и т.п. лучше хранить в отдельном файле и ни в коем случае не выкладывать в открытый доступ! Создадим в рабочей папке питоновский файл (например, `conf.py`) и запишем туда токен:
    
  `TOKEN = "..."`
  
В основной файл с кодом бота конфиги можно импортировать строчкой `import conf`, как любую библиотеку. Тогда переменная `TOKEN` будет доступна внутри основной программы как `conf.TOKEN`.

Если вы работаете в колабе, просто запустите следующую ячейку и вставьте в строку ввода ваш токен, а затем нажмите `Enter`.

In [76]:
with open("conf.py", "w", encoding="utf-8") as f:
    f.write('TOKEN = "%s"' % input())

## Онегин-бот

Отлично, теперь можно переходить к самому интересному — к написанию логики бота! Допустим, мы хотим, чтобы он умел:

1. Присылать пользователю случайные цитаты из определенного текста;
2. Делать полнотекстовый поиск и отправлять цитаты с заданным словом или фразой.

Для начала импортируем нужные библиотеки и напишем две вспомогательные функции:
1. `random_quote`, которая возвращает случайный элемент из списка.
2. `get_sentences`, которая читает текст из файла, разбививает его на предложения и возвращает список этих предложений. Обязательный аргумент — путь к текстовому файлу, необязательный — регулярное выражение для разбиения текста на предложения.

In [65]:
import re
import conf
import random
import telebot

In [70]:
def random_quote(sents):
    i = random.randint(0, len(sents)-1)
    return sents[i]

def get_sentences(path, regex=".*?[.!?]\s"):
    s = re.compile(regex, re.DOTALL)
    with open(path, "r", encoding="utf-8") as f:
        sents = s.findall(f.read())
    return sents

В качестве примера возьмем «Евгения Онегина» — уж у Пушкина-то точно найдутся строчки на все случаи жизни! А вообще можете использовать любые данные, которые вам нравятся — от текстов песен любимой группы до архива новостей местной газеты. Главное условие, чтобы это был файл формата `.txt` с кодировкой `utf-8`.

In [7]:
sents = get_sentences("onegin.txt")

Создаем экземпляр бота, используя наш секретный токен.

In [66]:
bot = telebot.TeleBot(conf.TOKEN)

Теперь напишем **обработчики сообщений** *(message handlers)* — специальные функции, которые позволят боту реагировать на сообщения. В общем случае обработчик сообщений выглядит так:

```python
# декоратор, который указывает, что это обработчик сообщений
@bot.message_handler(...)
# определение функции
def my_function(message):
    # код, который генерирует ответ
    reply = 'Привет!'
    # код для отправки сообщения в чат
    bot.send_message(message.chat.id, reply)
```

* `bot` — это имя переменной с нашим ботом;
* `...` — фильтр (то, на какие сообщения будет реагировать функция)

Фильтры бывают четырех типов:
* `content_types`, значение — массив строк, задающих тип контента (текст, аудио, файл, стикер и т.д., по умолчанию ['text'])
* `regexp`, значение — регулярное выражение (строка)
* `commands`, значение — массив строк, задающих команды (без знака /)
* `func`, значение — какая-либо функция

Что происходит, когда боту приходит сообщение, подходящее под несколько разных фильтров, т.е. несколько разных функций? В этом случае запускается та функция, которая написана в коде раньше других. [Примеры обработчиков](https://github.com/eternnoir/pyTelegramBotAPI#message-handlers) можно найти в документации библиотеки.

Для начала напишем функцию `send_quote`, которая по команде `\send` будет отправлять пользователю случайную цитату, т.е. случайное предложение из текста. Для этого внутри `send_quote` вызовем функцию `random_quote` и передадим ей в качестве аргумента переменную `sents`, в которой лежит разбитый на предложения текст романа.

In [68]:
@bot.message_handler(commands=['send'])
def send_quote(message):
    bot.send_message(message.chat.id, random_quote(sents))

Обработчик сообщений необязательно связан с какой-то командой. Например, можно написать такую функцию, которая будет реагировать на любое сообщение. Воспользуемся этим, чтобы реализовать поиск по тексту.

Будем считать любое сообщение (кроме команд) поисковым запросом и искать в тексте точное совпадение без учета регистра — примерно как Ctrl+F/Сmd+F в текстовом редакторе или браузере. Если результатов окажется несколько, то выберем один из них с помощью все той же функции `random_quote`. Если же не найдется ничего, сообщим об этом пользователю.

In [69]:
@bot.message_handler(func=lambda m: True)
def query_quote(message):
    query = message.text
    res = [sent for sent in sents if query.lower() in sent.lower()]
    if len(res) != 0:
        bot.send_message(message.chat.id, random_quote(res))
    else:
        bot.send_message(message.chat.id, "Об этом Пушкин не писал!")

Теперь сделаем так, чтобы наш бот постоянно спрашивал сервера телеграма, не пришло ли ему новых сообщений. Параметр `none_stop=True` означает, что при возникновении ошибок бот должен стараться не прекращать работу.

In [71]:
if __name__ == '__main__':
    bot.polling(none_stop=True)

Соберем весь код в одной ячейке. Чтобы бот заработал, нужно просто запустить ее, или же скопировать код в отдельный питоновский файл (например, `bot.py`) и запустить его в терминале командой `python bot.py`. Не забудьте, что во втором случае

1. в одной папке с `bot.py` должны лежать файлы `conf.py` и `onegin.txt` (или тот текст, с которым вы работаете);
2. в терминале вам нужно либо перейти в эту папку, либо указать полный путь к файлу `bot.py`;
3. на старых маках вместо `python` возможно придется написать `python3`.

<img src="./img/terminal.png" width="400" align="left">

In [1]:
import re
import conf
import random
import telebot

def get_sentences(path, regex=".*?[.!?]\s"):
    with open(path, "r", encoding="utf-8") as f:
        sents = re.findall(regex, f.read(), re.DOTALL)
    return sents

def random_quote(sents):
    i = random.randint(0, len(sents)-1)
    return sents[i]

bot = telebot.TeleBot(conf.TOKEN) 
sents = get_sentences("onegin.txt")

@bot.message_handler(commands=['send'])
def send_quote(message):
    bot.send_message(message.chat.id, random_quote(sents))
    
@bot.message_handler(func=lambda m: True) 
def query_quote(message):
    query = message.text
    res = [sent for sent in sents if query.lower() in sent.lower()]
    if len(res) != 0:
        bot.send_message(message.chat.id, random_quote(res))
    else:
        bot.send_message(message.chat.id, "Об этом Пушкин не писал!")

if __name__ == '__main__':
    bot.polling(none_stop=True)

Теперь нашего бота можно найти в телеграме и написать ему! Он будет доступен, пока запущена программа.

<img src="./img/send.png" width="550" align="left">
<img src="./img/search.png" width="550" align="left">
<img src="./img/not_found.png" width="550" align="left">


С помощью `@BotFather` можно настроить для нашего бота описание, список команд, аватарку и еще много чего.

**Список команд**

<img src="./img/setcommands.png" width="500" align="left">


Теперь, когда пользователь введет `\` в чате с ботом, он увидит выплывающий список команд:

<img src="./img/commands.png" width="500" align="left">

**Описание**

<img src="./img/setdescription.png" width="500" align="left">

Описание появится в самом начале чата с ботом.

<img src="./img/descr.png" width="500" align="left">